# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,papatowai,-46.5619,169.4708,11.88,89,100,3.34,NZ,1740955927
1,1,afaahiti,-17.7500,-149.2833,26.80,74,9,5.06,PF,1740955928
2,2,waitangi,-43.9535,-176.5597,17.68,94,93,2.68,NZ,1740955929
3,3,grytviken,-54.2811,-36.5092,3.42,80,23,4.52,GS,1740955930
4,4,greymouth,-42.4667,171.2000,19.83,79,93,2.50,NZ,1740955931


---
### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    x='Lng', 
    y='Lat', 
    geo=True, 
    size='Humidity', 
    scale=1,  # Adjust scale if needed
    tiles='OSM', 
    color='Humidity', 
    cmap='coolwarm', 
    frame_width=700, 
    frame_height=500,
    title="City Humidity Map"
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [21]:
# Narrow down cities that fit criteria and drop any results with null values
#I'm planning for a windsurf vacation so I'm looking for strong wind of 10.0 or more

# Drop any rows with null values
city_data_df.dropna(inplace=True)

# Display sample data
wind_cities =city_data_df[city_data_df["Wind Speed"]>10]

wind_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,38,edinburgh of the seven seas,-37.0676,-12.3116,18.68,85,81,10.88,SH,1740955890
3,42,longyearbyen,78.2186,15.6401,-1.09,74,20,12.35,SJ,1740955975
2,44,selfoss,63.9331,-20.9971,-1.09,69,100,14.47,IS,1740955977
5,91,rawson,-43.3002,-65.1023,30.16,48,0,11.83,AR,1740956029
0,171,keflavik,64.0049,-22.5624,-0.76,93,75,18.01,IS,1740956123
11,176,la jolla,32.8473,-117.2742,17.30,62,40,10.29,US,1740955905
9,226,bubaque,11.2833,-15.8333,23.52,67,4,10.63,GW,1740956187
10,325,kristiansund,63.1115,7.7320,7.36,81,75,10.29,NO,1740955911
1,403,klaksvik,62.2266,-6.5890,5.90,70,78,15.38,FO,1740956396
6,468,floro,61.5996,5.0328,5.98,93,100,11.83,NO,1740956470


### Step 3: Create a new DataFrame called `hotel_df`.

In [28]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = wind_cities[["City",'Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.loc[:,"Hotel Name"] =""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
7,edinburgh of the seven seas,SH,-37.0676,-12.3116,85,
3,longyearbyen,SJ,78.2186,15.6401,74,
2,selfoss,IS,63.9331,-20.9971,69,
5,rawson,AR,-43.3002,-65.1023,48,
0,keflavik,IS,64.0049,-22.5624,93,
11,la jolla,US,32.8473,-117.2742,62,
9,bubaque,GW,11.2833,-15.8333,67,
10,kristiansund,NO,63.1115,7.7320,81,
1,klaksvik,FO,62.2266,-6.5890,70,
6,floro,NO,61.5996,5.0328,93,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [37]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "limit": 1,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat,lng =row["Lat"],row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response =requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")



# Display sample data
# hotel_df

Starting hotel search
edinburgh of the seven seas - nearest hotel: No hotel found
longyearbyen - nearest hotel: The Vault
selfoss - nearest hotel: Miðgarður
rawson - nearest hotel: Hotel Deportivo
keflavik - nearest hotel: Núpan Deluxe
la jolla - nearest hotel: Grande Colonial La Jolla
bubaque - nearest hotel: Chez Julio
kristiansund - nearest hotel: Comfort Hotel Fosna
klaksvik - nearest hotel: Hotel Klaksvik
floro - nearest hotel: Comfort Hotel Victoria Florø
nemuro - nearest hotel: お宿エクハシ
kijang - nearest hotel: 힐튼호텔 부산


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
wind_hotels = hotel_df.hvplot.points(
    x='Lng', 
    y='Lat', 
    geo=True, 
    size='Humidity', 
    scale=1, 
    tiles='OSM', 
    color='Humidity', 
    cmap='coolwarm', 
    frame_width=700, 
    frame_height=500,
    title="Wind Speed Hotels"
)

# Display the map
city_map

# Display the map
city_map



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)